<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/Plotly_with_kwargs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.graph_objects as go
import pandas as pd # Assuming you use pandas for results_df

In [13]:
# @title def trace_from_path (Universal with **kwargs)
def trace_from_path(
    results_df,
    **kwargs          # Capture arbitrary Scatter3d keyword arguments
    ):
    """
    Create 3D scatter trace of crystallization pathway, allowing any
    go.Scatter3d option to be passed via keyword arguments.

    Args:
        results_df (pd.DataFrame): DataFrame containing crystallization results.
                                   Must have columns 'x', 'y', 'z',
                                   'final_solution_mass', 'nacl_mass',
                                   'carnallite_mass'.
        **kwargs: Keyword arguments passed directly to the go.Scatter3d constructor.
                  These will override the function's defaults if the same key
                  (e.g., 'mode', 'marker', 'line', 'visible', 'opacity') is provided.
                  See Plotly documentation for go.Scatter3d for all available options.

    Returns:
        go.Scatter3d: Plotly Scatter3d trace for 3D visualization.
    """
    # --- Calculate necessary data from DataFrame ---
    x_data = results_df['x']
    y_data = results_df['y']
    z_data = results_df['z']
    hover_text = [f'Step {i}<br>Solution mass: {e:.1f}g<br>NaCl: {n:.1f}g<br>Carnallite: {c:.1f}g'
                  for i, e, n, c in zip(results_df.index,
                                      results_df['final_solution_mass'],
                                      results_df['nacl_mass'],
                                      results_df['carnallite_mass'])]

    # --- Define the base/default options for this specific trace type ---
    # These are applied if not overridden by kwargs
    trace_options = {
        'mode': 'lines+markers',
        'marker': dict(
            size=5,
            color=z_data,  # Default to coloring by z-value
            colorscale='Viridis',
        ),
        'line': dict(
            width=3
            # Line color often defaults to marker color or a standard sequence
        ),
        'hoverinfo': 'text' # Essential because we provide custom text
        # Add any other essential defaults specific to this function's purpose
    }

    # --- Update defaults with any user-provided kwargs ---
    # If kwargs contains 'mode', 'marker', 'line', 'opacity', etc.,
    # it will overwrite the values defined in trace_options above.
    trace_options.update(kwargs)

    # --- Create the Scatter3d trace ---
    # Pass the calculated data and the (potentially updated) options
    trace = go.Scatter3d(
        x=x_data,
        y=y_data,
        z=z_data,
        text=hover_text,
        **trace_options # Unpack mode, marker, line, hoverinfo, and any user kwargs
    )

    return trace

In [14]:
# --- Example Usage ---

# Create some dummy data (same as before)
data = {
    'x': [10, 12, 15, 13, 11],
    'y': [20, 22, 25, 23, 21],
    'z': [30, 35, 40, 38, 32],
    'final_solution_mass': [100, 95, 90, 92, 98],
    'nacl_mass': [5, 4, 3, 3.5, 4.5],
    'carnallite_mass': [2, 3, 4, 3.8, 2.5]
}
results_df_example = pd.DataFrame(data)

# 1. Default trace (lines+markers, z-based color)
trace1 = trace_from_path(results_df_example, name='Default')

# 2. Override mode and set a simple red marker via kwargs
trace2 = trace_from_path(
    results_df_example,
    name='Kwargs: Markers Only, Red',
    mode='markers', # Override default mode
    marker=dict(color='red', size=8) # Override default marker dict entirely
)

# 3. Override line properties using kwargs
trace3 = trace_from_path(
    results_df_example,
    name='Kwargs: Blue Dashed Line',
    mode='lines', # Override default mode
    line=dict(color='blue', width=5, dash='dash') # Override default line dict
)

# 4. Override marker color and symbol, keep default line width/color
trace4 = trace_from_path(
    results_df_example,
    name='Kwargs: Green Diamonds',
    marker=dict(color='green', size=7, symbol='diamond') # Override marker dict
    # mode='lines+markers' (default) is used
    # line=dict(width=3) (default) is used
)

# 5. Pass another Scatter3d argument like 'opacity'
trace5 = trace_from_path(
    results_df_example,
    name='Semi-Transparent',
    opacity=0.6 # Pass a standard Scatter3d arg via kwargs
)

In [15]:
# To display these (requires plotly installed)
fig = go.Figure(data=[trace1, trace2, trace3, trace4, trace5])
fig.update_layout(title='Customizable Crystallization Paths (Universal **kwargs)',
                  scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
fig.show() # Uncomment to display the plot